Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/track-and-monitor-experiments/using-mlflow/train-remote/train-remote.png)

## Use MLflow with Azure Machine Learning for Remote Training Run

This example shows you how to use MLflow tracking APIs together with Azure Machine Learning services for storing your metrics and artifacts, from local Notebook run. You'll learn how to:

 1. Set up MLflow tracking URI so as to use Azure ML
 2. Create experiment
 3. Train a model on Machine Learning Compute while logging metrics and artifacts
 4. View your experiment within your Azure ML Workspace in Azure Portal.

## Prerequisites

Make sure you have completed the [Configuration](../../../configuration.ipnyb) notebook to set up your Azure Machine Learning workspace and ensure other common prerequisites are met.

## Set-up

Check Azure ML SDK version installed on your computer, and then connect to your Workspace.

In [ ]:
# Check core SDK version number
import azureml.core
from azureml.core import Workspace, Experiment

print("SDK version:", azureml.core.VERSION)

ws = Workspace.from_config()

Let's also create a Machine Learning Compute cluster for submitting the remote run. 

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=2)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

### Create Azure ML Experiment

The following steps show how to submit a training Python script to a cluster as an Azure ML run, while logging happens through MLflow APIs to your Azure ML Workspace. Let's first create an experiment to hold the training runs.

In [ ]:
from azureml.core import Experiment

experiment_name = "RemoteTrain-with-mlflow-sample"
exp = Experiment(workspace=ws, name=experiment_name)

### Instrument remote training script using MLflow

Let's use [*train_diabetes.py*](train_diabetes.py) to train a regression model against diabetes dataset as the example. Note that the training script uses mlflow.start_run() to start logging, and then logs metrics, saves the trained scikit-learn model, and saves a plot as an artifact.

Run following command to view the script file. Notice the mlflow logging statements, and also notice that the script doesn't have explicit dependencies on azureml library.

In [ ]:
training_script = 'train_diabetes.py'
with open(training_script, 'r') as f:
    print(f.read())

### Submit Run to Cluster 

Let's submit the run to cluster. When running on the remote cluster as submitted run, Azure ML sets the MLflow tracking URI to point to your Azure ML Workspace, so that the metrics and artifacts are automatically logged there.

Note that you have to specify the packages your script depends on, including *azureml-mlflow* that implicitly enables the MLflow logging to Azure ML. 

First, create a environment with Docker enable and required package dependencies specified.

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment(name="mlflow-env")

# Specify conda dependencies with scikit-learn and temporary pointers to mlflow extensions
cd = CondaDependencies.create(
    conda_packages=["scikit-learn", "matplotlib"],
    pip_packages=["azureml-mlflow", "pandas", "numpy"]
    )

env.python.conda_dependencies = cd

Next, specify a script run configuration that includes the training script, environment and CPU cluster created earlier.

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=".",
                      script=training_script,
                      compute_target=cpu_cluster,
                      environment=env)

Finally, submit the run. Note that the first instance of the run typically takes longer as the Docker-based environment is created, several minutes. Subsequent runs reuse the image and are faster.

In [ ]:
run = exp.submit(src)
run.wait_for_completion(show_output=True)

You can navigate to your Azure ML Workspace at Azure Portal to view the run metrics and artifacts. 

In [ ]:
run

You can also get the metrics and bring them to your local notebook, and view the details of the run.

In [ ]:
run.get_metrics()

In [ ]:
run.get_details()

### Next steps

 * [Deploy the model as a web service](../deploy-model/deploy-model.ipynb)
 * [Learn more about Azure Machine Learning compute options](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets)